In [1]:
from google.colab import drive
drive.mount("/content/drive")
%cd /content/drive/MyDrive/BikeCSDI
!git clone https://github.com/nowiley/ergo-bike.git

Mounted at /content/drive
/content/drive/MyDrive/BikeCSDI
fatal: destination path 'ergo-bike' already exists and is not an empty directory.


In [2]:
import pandas as pd
import numpy as np

In [3]:
bikes = pd.read_csv("new_colored_biked.csv")
for i in bikes.columns:
  print(i)

Unnamed: 0
BB textfield
Seat tube length
Stack
Head tube upper extension2
Seat angle
CS textfield
FCD textfield
Seat tube extension2
Head tube lower extension2
Head angle
Saddle height
ERD rear
Wheel width rear
Dropout spacing
SPOKES composite front
SPOKES front
Wheel diameter front
SAME_SIZED_FRONT_AND_REAR
BSD front
Wheel diameter rear
SPOKES rear
Wheel width front
SPOKES composite rear
ERD front
BSD rear
SBLADEW front
SBLADEW rear
FORK0R
FORK0L
Saddle P
Saddle thickness
Saddle angle
Saddle J
Saddle H
Saddle E
SADDLETIPtoMIDDLE
Saddle length
Seatpost setback
Seatpost LENGTH
Lower stack height
Upper stack height
Headset spacers
Stem angle
CLAMPOFFSET
StemX
StemY
StemG
Collar height
Stem length
AEROF
Hand horizontal
Pad vertical
Display AEROBARS
Hand vertical
HBARRISE
MtnBar angle
Mountain bar sweep
MTNDROP
HBARTHETA
Mountain bar rise
Handlebar angle
Bullhorn reach
BULLTHETA
Road bar reach
BULLRADIUS2
Road bar drop
Brake lever position
Bullhorn drop
Bullhorn rise
Bullhorn risy
Bullhorn

In [4]:
print(bikes["Head angle"][0:10])

0    73.000000
1    71.500000
2    72.000000
3    70.500000
4    72.100000
5    80.600000
6    69.534836
7    70.000000
8    73.000000
9    72.000000
Name: Head angle, dtype: float64


DT Length

In [5]:
bikes["DT Length"] = 0
for i in range(100):
  bbd = bikes["BB textfield"][i]
  fcd = bikes["FCD textfield"][i]
  fty = bbd
  ftx = np.sqrt(fty ** 2 + fcd ** 2)
  x = bikes["FORK0R"][i]
  fkl = bikes["FORK0L"][i]
  htlx = bikes["Head tube lower extension2"][i]
  lsth = bikes["Lower stack height"][i]
  y = fkl + htlx + lsth
  ha = bikes["Head angle"][i] * np.pi / 180
  dtx = ftx - y * np.cos(ha) - x * np.sin(ha)
  dty = fty + y * np.sin(ha) + x * np.cos(ha)
  bikes.loc[i, "DT Length"] = np.sqrt(dtx ** 2 + dty ** 2)


In [6]:
%cd ergo-bike/pythonversion

/content/drive/MyDrive/BikeCSDI/ergo-bike/pythonversion


In [7]:
from angles import *
from interfacepoints import interface_points

In [8]:
LL = 19
UL = 15.5
TL = 21
AL = 24
FL = 5.5
AA = deg_to_r(107)

#正确的数量级应与上面的数据相近，但输出都是三位数了

arm_angle = 150
body = np.array([[LL, UL, TL, AL, FL, AA]]).T
output = pd.DataFrame(columns=['Seat X', 'Seat Y', 'Handle X', 'Handle Y', 'Crank Length', 'Lower Leg', 'Upper Leg', 'Torso Length', 'Arm Length', 'Foot Length', 'Ankle Angle', 'Knee Extension Angle', 'Back Angle', 'Armpit to Wrist Angle', 'Use Case'])
use = ["road", "mtb", "commute"]

bike_features = ["DT Length", 100, "Head angle", "Head tube lower extension2", "Stack", "Seat tube length", "Seat angle", "Seatpost LENGTH", "Saddle height", "Stem length", "Stem angle", "Headset spacers", "Crank length", "Handlebar style"]

for i in range(100):
  bike = np.array([[bikes[f][i] if f in bikes.columns else f for f in bike_features]])
  print([[bikes[f][i] if f in bikes.columns else f for f in bike_features]])
  bike = interface_points(bike).T / 25.4
  print(bike)
  kea, ba, awa = all_angles(bike, body, arm_angle)
  if not ba:
    continue
  best = None
  min = np.inf
  for u in use:
    ob, ow = USE_DICT[u]['opt_back_angle'][0], USE_DICT[u]['opt_awrist_angle'][0]
    d = (ba-ob)**2 + (awa-ow)**2
    if d < min:
      best = u
      min = d
  output.loc[len(output.index)] = [bike[j, 0] for j in range(5)] + [LL, UL, TL, AL, FL, AA] + [kea, ba, awa] + [best]

output.to_csv("ergo-output.csv")

# 输出的长行是13个特征，和它的示例数量级相近，也跟ground truth相近。

[[664.0206902489724, 100, 73.0, 50.0, 565.6, 588.7, 72.5, 300.0, 768.0, 120.0, -10.0, 15.0, 172.5, 0]]
[[23.36231374]
 [27.08244479]
 [ 9.09220685]
 [28.83679599]
 [ 6.79133858]]


/content/drive/MyDrive/BikeCSDI/ergo-bike/pythonversion/angles.py:88: RuntimeWarning: invalid value encountered in arccos
  alpha_1 = np.arccos((x_1**2 - UL_s - x_2**2) / (-2 * UL * x_2))
/content/drive/MyDrive/BikeCSDI/ergo-bike/pythonversion/angles.py:88: RuntimeWarning: invalid value encountered in arccos
  alpha_1 = np.arccos((x_1**2 - UL_s - x_2**2) / (-2 * UL * x_2))
/content/drive/MyDrive/BikeCSDI/ergo-bike/pythonversion/angles.py:88: RuntimeWarning: invalid value encountered in arccos
  alpha_1 = np.arccos((x_1**2 - UL_s - x_2**2) / (-2 * UL * x_2))
/content/drive/MyDrive/BikeCSDI/ergo-bike/pythonversion/angles.py:88: RuntimeWarning: invalid value encountered in arccos
  alpha_1 = np.arccos((x_1**2 - UL_s - x_2**2) / (-2 * UL * x_2))
/content/drive/MyDrive/BikeCSDI/ergo-bike/pythonversion/angles.py:88: RuntimeWarning: invalid value encountered in arccos
  alpha_1 = np.arccos((x_1**2 - UL_s - x_2**2) / (-2 * UL * x_2))
/content/drive/MyDrive/BikeCSDI/ergo-bike/pythonversion/angl

[[572.4919500516429, 100, 71.5, 23.1, 565.6, 265.0, 71.5, 100.0, 330.0, 40.0, 0.0, 15.0, 170.0, 1]]
[[10.60031241]
 [24.63418776]
 [ 4.12246207]
 [12.3207404 ]
 [ 6.69291339]]
[[647.1036536882136, 100, 72.0, 35.9, 565.6, 562.0, 74.0, 300.0, 763.0, 80.0, 0.0, 10.0, 175.0, 1]]
[[16.82850554]
 [24.92594105]
 [ 8.27997254]
 [28.87569583]
 [ 6.88976378]]
[[573.9040929925656, 100, 70.5, 33.9, 565.6, 460.0, 73.5, 140.0, 600.0, 90.0, -20.0, 30.0, 165.0, 0]]
[[15.80632185]
 [27.72121011]
 [ 6.70902389]
 [22.64928508]
 [ 6.49606299]]
[[636.1792967995458, 100, 72.1, 54.3, 565.6, 300.0, 72.5, 250.0, 350.0, 50.0, 0.0, 15.0, 175.0, 1]]
[[14.23830667]
 [24.74596658]
 [ 4.14358385]
 [13.14176901]
 [ 6.88976378]]
[[601.509396892262, 100, 80.6, 25.0, 565.6, 315.0, 71.6, 300.0, 350.0, 50.0, 0.0, 15.0, 170.0, 1]]
[[13.99682303]
 [24.53129552]
 [ 4.3494946 ]
 [13.07506315]
 [ 6.69291339]]
[[688.291607604739, 100, 69.53483637778452, 50.0, 565.6, 535.0, 72.03483637778452, 400.0, 755.0, 90.0, 6.0, 30.0, 175.0

/content/drive/MyDrive/BikeCSDI/ergo-bike/pythonversion/angles.py:88: RuntimeWarning: invalid value encountered in arccos
  alpha_1 = np.arccos((x_1**2 - UL_s - x_2**2) / (-2 * UL * x_2))
/content/drive/MyDrive/BikeCSDI/ergo-bike/pythonversion/angles.py:88: RuntimeWarning: invalid value encountered in arccos
  alpha_1 = np.arccos((x_1**2 - UL_s - x_2**2) / (-2 * UL * x_2))
/content/drive/MyDrive/BikeCSDI/ergo-bike/pythonversion/angles.py:88: RuntimeWarning: invalid value encountered in arccos
  alpha_1 = np.arccos((x_1**2 - UL_s - x_2**2) / (-2 * UL * x_2))
/content/drive/MyDrive/BikeCSDI/ergo-bike/pythonversion/angles.py:88: RuntimeWarning: invalid value encountered in arccos
  alpha_1 = np.arccos((x_1**2 - UL_s - x_2**2) / (-2 * UL * x_2))
/content/drive/MyDrive/BikeCSDI/ergo-bike/pythonversion/angles.py:88: RuntimeWarning: invalid value encountered in arccos
  alpha_1 = np.arccos((x_1**2 - UL_s - x_2**2) / (-2 * UL * x_2))
/content/drive/MyDrive/BikeCSDI/ergo-bike/pythonversion/angl

[[554.7817601166938, 100, 40.0, 46.3, 565.6, 450.9, 40.0, 270.0, 600.0, 80.0, 0.0, 10.0, 170.0, 1]]
[[ 4.57613503]
 [25.81925189]
 [18.09553803]
 [15.18395928]
 [ 6.69291339]]
[[562.3684832667249, 100, 30.0, 46.3, 565.6, 550.0, 25.0, 150.0, 650.0, 80.0, 0.0, 10.0, 172.5, 1]]
[[ 3.77123513]
 [25.88118237]
 [23.19291581]
 [10.81503426]
 [ 6.79133858]]
[[623.4443656860658, 100, 73.0, 44.0, 565.6, 540.0, 74.5, 250.0, 705.0, 100.0, -10.0, 15.0, 170.0, 2]]
[[20.22695763]
 [26.33127117]
 [ 7.41744312]
 [26.74643581]
 [ 6.69291339]]
[[685.3063322624585, 100, 74.035, 64.5, 565.6, 580.0, 72.535, 300.0, 764.0, 100.0, -4.0, 20.0, 172.5, 0]]
[[23.69083356]
 [26.4812303 ]
 [ 9.02732627]
 [28.69212418]
 [ 6.79133858]]
[[514.566274633877, 100, 74.0, 40.8, 565.6, 480.1, 72.1, 200.0, 618.0, 100.0, -5.0, 10.0, 165.0, 1]]
[[ 4.80937186]
 [25.38163666]
 [ 7.47820432]
 [23.1529662 ]
 [ 6.49606299]]
[[623.4443656860658, 100, 73.0, 44.0, 565.6, 500.0, 74.5, 250.0, 705.0, 100.0, -10.0, 15.0, 170.0, 1]]
[[15.50

/content/drive/MyDrive/BikeCSDI/ergo-bike/pythonversion/angles.py:88: RuntimeWarning: invalid value encountered in arccos
  alpha_1 = np.arccos((x_1**2 - UL_s - x_2**2) / (-2 * UL * x_2))
/content/drive/MyDrive/BikeCSDI/ergo-bike/pythonversion/angles.py:88: RuntimeWarning: invalid value encountered in arccos
  alpha_1 = np.arccos((x_1**2 - UL_s - x_2**2) / (-2 * UL * x_2))
/content/drive/MyDrive/BikeCSDI/ergo-bike/pythonversion/angles.py:88: RuntimeWarning: invalid value encountered in arccos
  alpha_1 = np.arccos((x_1**2 - UL_s - x_2**2) / (-2 * UL * x_2))
/content/drive/MyDrive/BikeCSDI/ergo-bike/pythonversion/angles.py:88: RuntimeWarning: invalid value encountered in arccos
  alpha_1 = np.arccos((x_1**2 - UL_s - x_2**2) / (-2 * UL * x_2))
/content/drive/MyDrive/BikeCSDI/ergo-bike/pythonversion/angles.py:88: RuntimeWarning: invalid value encountered in arccos
  alpha_1 = np.arccos((x_1**2 - UL_s - x_2**2) / (-2 * UL * x_2))
/content/drive/MyDrive/BikeCSDI/ergo-bike/pythonversion/angl

[[634.3747857132333, 100, 72.0, 39.7, 565.6, 500.0, 74.0, 330.0, 600.0, 110.0, -10.0, 50.0, 167.0, 1]]
[[16.21436693]
 [27.48353054]
 [ 6.51111864]
 [22.7069692 ]
 [ 6.57480315]]
[[634.3747857132333, 100, 72.0, 39.7, 565.6, 500.0, 74.0, 330.0, 600.0, 110.0, -10.0, 50.0, 167.0, 1]]
[[16.21436693]
 [27.48353054]
 [ 6.51111864]
 [22.7069692 ]
 [ 6.57480315]]
[[633.1337857977499, 100, 72.0, 39.6, 565.6, 500.0, 74.0, 330.0, 600.0, 110.0, -10.0, 50.0, 167.0, 1]]
[[16.13639988]
 [27.48353054]
 [ 6.51111864]
 [22.7069692 ]
 [ 6.57480315]]
[[740.4227763906955, 100, 72.0, 40.0, 565.6, 535.0, 73.0, 400.0, 760.0, 80.0, -10.0, 10.0, 172.5, 1]]
[[21.90351287]
 [25.43130977]
 [ 8.74812975]
 [28.61384309]
 [ 6.79133858]]
[[649.5578111348416, 100, 72.0, 47.6, 565.6, 567.0, 74.0, 300.0, 718.0, 100.0, -5.0, 10.0, 170.0, 1]]
[[17.19722062]
 [25.49097071]
 [ 7.79163864]
 [27.17267314]
 [ 6.69291339]]
[[616.9746132745836, 100, 71.5, 45.4, 565.6, 291.1, 74.0, 115.0, 365.0, 45.0, -5.0, 7.0, 175.0, 1]]
[[13.07

/content/drive/MyDrive/BikeCSDI/ergo-bike/pythonversion/angles.py:88: RuntimeWarning: invalid value encountered in arccos
  alpha_1 = np.arccos((x_1**2 - UL_s - x_2**2) / (-2 * UL * x_2))
/content/drive/MyDrive/BikeCSDI/ergo-bike/pythonversion/angles.py:88: RuntimeWarning: invalid value encountered in arccos
  alpha_1 = np.arccos((x_1**2 - UL_s - x_2**2) / (-2 * UL * x_2))
/content/drive/MyDrive/BikeCSDI/ergo-bike/pythonversion/angles.py:88: RuntimeWarning: invalid value encountered in arccos
  alpha_1 = np.arccos((x_1**2 - UL_s - x_2**2) / (-2 * UL * x_2))
/content/drive/MyDrive/BikeCSDI/ergo-bike/pythonversion/angles.py:88: RuntimeWarning: invalid value encountered in arccos
  alpha_1 = np.arccos((x_1**2 - UL_s - x_2**2) / (-2 * UL * x_2))
/content/drive/MyDrive/BikeCSDI/ergo-bike/pythonversion/angles.py:88: RuntimeWarning: invalid value encountered in arccos
  alpha_1 = np.arccos((x_1**2 - UL_s - x_2**2) / (-2 * UL * x_2))
/content/drive/MyDrive/BikeCSDI/ergo-bike/pythonversion/angl

[[684.3796922063729, 100, 70.0, 48.0, 565.6, 505.0, 72.5, 400.0, 730.0, 80.0, 6.0, 20.0, 175.0, 1]]
[[18.49442818]
 [25.06444275]
 [ 8.64233203]
 [27.40997536]
 [ 6.88976378]]
[[654.7202094275177, 100, 69.5, 50.0, 565.6, 300.0, 73.0, 300.0, 380.0, 30.0, 5.0, 10.0, 172.5, 1]]
[[14.76629052]
 [24.24281102]
 [ 4.37406487]
 [14.30692155]
 [ 6.79133858]]
[[676.5120728599138, 100, 70.5, 48.0, 565.6, 505.0, 72.5, 400.0, 730.0, 80.0, 6.0, 20.0, 170.0, 1]]
[[18.06659041]
 [25.04413036]
 [ 8.64233203]
 [27.40997536]
 [ 6.69291339]]
[[681.2732002461984, 100, 68.5, 55.0, 565.6, 505.0, 72.5, 400.0, 730.0, 65.0, 0.0, 20.0, 175.0, 1]]
[[17.30440779]
 [25.22029709]
 [ 8.64233203]
 [27.40997536]
 [ 6.88976378]]
[[721.1815483495776, 100, 71.0, 60.5, 565.6, 505.0, 73.5, 400.0, 735.0, 80.0, -6.0, 5.0, 175.0, 1]]
[[20.3776798 ]
 [25.08780331]
 [ 8.21855427]
 [27.74537422]
 [ 6.88976378]]
[[707.433390932504, 100, 71.5, 59.2, 565.6, 570.0, 73.0, 330.0, 730.0, 90.0, -6.0, 10.0, 172.5, 1]]
[[19.9496071 ]
 [25.